# 1. Mengatur Direktori/Path

In [1]:
apimodel_path = "C:\\Users\\Ican\\Tensorflow\\models"
scripts_path = "C:\\Users\\Ican\\Tensorflow\\scripts"
workspace_path = "C:\\Users\\Ican\\Tensorflow\\TA"
annotation_path = f"{workspace_path}\\annotations"
custom_model_path = f"{workspace_path}\\models"
image_path = f"{workspace_path}\\images"
pretrained_model_path = f"{workspace_path}\\pre-trained-models"
pretrained_model = "efficientdet_d1_coco17_tpu-32"
custom_model_name = "efficientdet_d1"
checkpoint_path = f"{custom_model_path}\\{custom_model_name}"
config_path = f"{custom_model_path}\\{custom_model_name}\\pipeline.config"

# 2. Membuat label map

In [2]:
labels = [{"name": "person", "id" : 1}]

In [3]:
labels

[{'name': 'person', 'id': 1}]

In [8]:
with open(f"{annotation_path}\\label_map.pbtxt", "w") as f:
    for label in labels:
        f.write("item{\n")
        f.write(f"      id:{label['id']}" + "\n")
        f.write(f"      name:'{label['name']}'"+ "\n")
        f.write("}\n")

# 3. Membuat TF records

In [9]:
!python {scripts_path + "\\preprocessing\\generate_tfrecord.py"} -x {image_path + "\\train"} -l{annotation_path + "\\label_map.pbtxt"} -o {annotation_path + "\\train.tfrecord"}
!python {scripts_path + "\\preprocessing\\generate_tfrecord.py"} -x {image_path + "\\test"} -l{annotation_path + "\\label_map.pbtxt"} -o {annotation_path + "\\test.tfrecord"}

Successfully created the TFRecord file: C:\Users\Ican\Tensorflow\TA\annotations\train.tfrecord
Successfully created the TFRecord file: C:\Users\Ican\Tensorflow\TA\annotations\test.tfrecord


# 4. Clone tensorflow-api dan download pre-trained models

clone tensorflow-apinya diletakan di folder Main:

!cd Main && git clone https://github.com/tensorflow/models

langkah selanjutnya dari tensorflow-api dapat dilihat pada web berikut : https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/install.html

pre-trained model dapat didownload di: https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md


# 5. Copy config model dari pre-trained model ke folder models yang ada di workspace

In [3]:
!mkdir {checkpoint_path}
import shutil
shutil.copy(f"{pretrained_model_path}\\{pretrained_model}\\pipeline.config", f"{checkpoint_path}")

'C:\\Users\\Ican\\Tensorflow\\TA\\models\\efficientdet_d1\\pipeline.config'

# 6. Update file config yang dicopy tadi Untuk Transfer Learning

In [2]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [5]:
config = config_util.get_configs_from_pipeline_file(config_path)

In [6]:
config

{'model': ssd {
   num_classes: 90
   image_resizer {
     keep_aspect_ratio_resizer {
       min_dimension: 640
       max_dimension: 640
       pad_to_max_dimension: true
     }
   }
   feature_extractor {
     type: "ssd_efficientnet-b1_bifpn_keras"
     conv_hyperparams {
       regularizer {
         l2_regularizer {
           weight: 3.9999998989515007e-05
         }
       }
       initializer {
         truncated_normal_initializer {
           mean: 0.0
           stddev: 0.029999999329447746
         }
       }
       activation: SWISH
       batch_norm {
         decay: 0.9900000095367432
         scale: true
         epsilon: 0.0010000000474974513
       }
       force_use_bias: true
     }
     bifpn {
       min_level: 3
       max_level: 7
       num_iterations: 4
       num_filters: 88
     }
   }
   box_coder {
     faster_rcnn_box_coder {
       y_scale: 1.0
       x_scale: 1.0
       height_scale: 1.0
       width_scale: 1.0
     }
   }
   matcher {
     argmax_matc

In [7]:
# Update data config
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(config_path, "r") as f:
    proto_str = f.read()
    text_format.Merge(proto_str, pipeline_config)

In [8]:
pipeline_config.model.ssd.num_classes = 1
pipeline_config.train_config.batch_size = int(128/64)
pipeline_config.train_config.optimizer.momentum_optimizer.learning_rate.cosine_decay_learning_rate.learning_rate_base = 0.07999999821186066/64
pipeline_config.train_config.optimizer.momentum_optimizer.learning_rate.cosine_decay_learning_rate.warmup_learning_rate = 0.0010000000474974513/64
pipeline_config.train_config.fine_tune_checkpoint = f"{pretrained_model_path}/{pretrained_model}/checkpoint/ckpt-0"
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_config.use_bfloat16 = False
pipeline_config.train_input_reader.label_map_path = f"{annotation_path}/label_map.pbtxt"
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [f"{annotation_path}/train.tfrecord"]
pipeline_config.eval_input_reader[0].label_map_path = f"{annotation_path}/label_map.pbtxt"
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[:] = [f"{annotation_path}/test.tfrecord"]

In [9]:
pipeline_config

model {
  ssd {
    num_classes: 1
    image_resizer {
      keep_aspect_ratio_resizer {
        min_dimension: 640
        max_dimension: 640
        pad_to_max_dimension: true
      }
    }
    feature_extractor {
      type: "ssd_efficientnet-b1_bifpn_keras"
      conv_hyperparams {
        regularizer {
          l2_regularizer {
            weight: 3.9999998989515007e-05
          }
        }
        initializer {
          truncated_normal_initializer {
            mean: 0.0
            stddev: 0.029999999329447746
          }
        }
        activation: SWISH
        batch_norm {
          decay: 0.9900000095367432
          scale: true
          epsilon: 0.0010000000474974513
        }
        force_use_bias: true
      }
      bifpn {
        min_level: 3
        max_level: 7
        num_iterations: 4
        num_filters: 88
      }
    }
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 1.0
        x_scale: 1.0
        height_scale: 1.0
        width_scale: 1.0

In [10]:
#save dan replace config baru
config_text = text_format.MessageToString(pipeline_config)
with tf.io.gfile.GFile(config_path, "wb") as f:
    f.write(config_text)

# 7. Training Model

In [11]:
print(f"python {apimodel_path}/research/object_detection/model_main_tf2.py --model_dir={checkpoint_path} --pipeline_config_path={config_path} --num_train_steps=10000")

python C:\Users\Ican\Tensorflow\models/research/object_detection/model_main_tf2.py --model_dir=C:\Users\Ican\Tensorflow\TA\models\efficientdet_d1 --pipeline_config_path=C:\Users\Ican\Tensorflow\TA\models\efficientdet_d1\pipeline.config --num_train_steps=10000


Tanpa Jumlah Step yang Diatur

In [12]:
print(f"python {apimodel_path}/research/object_detection/model_main_tf2.py --model_dir={checkpoint_path} --pipeline_config_path={config_path}")

python C:\Users\Ican\Tensorflow\models/research/object_detection/model_main_tf2.py --model_dir=C:\Users\Ican\Tensorflow\TA\models\efficientdet_d1 --pipeline_config_path=C:\Users\Ican\Tensorflow\TA\models\efficientdet_d1\pipeline.config


Jika Ingin mengevaluasi Model Menggunakan COCO evaluation metrics (Jika proses train tidak pakai num_steps, maka pada evaluasi juga tidak dipakai)

In [3]:
print(f"python {apimodel_path}/research/object_detection/model_main_tf2.py --model_dir={checkpoint_path} --pipeline_config_path={config_path} --checkpoint_dir={checkpoint_path}")

python C:\Users\Ican\Tensorflow\models/research/object_detection/model_main_tf2.py --model_dir=C:\Users\Ican\Tensorflow\TA\models\efficientdet_d1 --pipeline_config_path=C:\Users\Ican\Tensorflow\TA\models\efficientdet_d1\pipeline.config --checkpoint_dir=C:\Users\Ican\Tensorflow\TA\models\efficientdet_d1


Monitoring di Tensorboard

In [5]:
print(f"tensorboard --logdir={checkpoint_path}")

tensorboard --logdir=C:\Users\Ican\Tensorflow\TA\models\efficientdet_d1


copy dan paste hasil print diatas ke cmd yang telah diganti direktorinya